# Table of Contents

* [<font size = "4"> 1.  **Introduction**](#chapter1)
    * [1.1 StatsBomb API Information](#section_1_1)
    * [1.2 Notebook Instructions](#section_1_2)
    * [1.3 Load Packages](#section_1_3)
    * [1.4 StatsBomb Credentials](#section_1_4)
    * [1.5 Optional View of DataFrames](#section_1_5)

    <br>
    
* [<font size = "4"> 2. **Extraction**](#chapter2)
    * [2.1 Extract Competitions Data](#section_2_1)
    * [2.2 Choose Competition, Season & Time Range](#section_2_2)
    * [2.3 Extract Matches Data](#section_2_3)
    * [2.4 Extract Team Season Data](#section_2_4)    
    * [2.5 Extract Player Season Data](#section_2_5)      
    * [2.6 Extract Player Match Data](#section_2_6)
    * [2.7 Extract Events Data](#section_2_7)
    * [2.8 Extract Lineups Data](#section_2_8)

    <br>
    
* [<font size = "4"> 3. **Transformation**](#chapter3)
    * [3.1 Transform Competitions Data](#section_3_1)
    * [3.2 Transform Matches Data](#section_3_2)
    * [3.3 Transform Team Season Data](#section_3_3)    
    * [3.4 Transform Player Season Data](#section_3_4)      
    * [3.5 Transform Player Match Data](#section_3_5)
        * [3.5.1 Team Match Data](#section_3_5_1)
    * [3.6 Transform Events Data](#section_3_6)
        * [3.6.1 Events](#section_3_6_1)
        * [3.6.2 Shot Freeze Frame](#section_3_6_2)    
    * [3.7 Transform Lineups Data](#section_3_7)
        * [3.7.1 Lineups](#section_3_7_1)
        * [3.7.2 Formations](#section_3_7_2)
    
    <br>
    
* [<font size = "4"> 4. **Load**](#chapter4)
    * [4.1 Option A: Export to .csv](#section_4_1)
    * [4.2 Option B: Load to layer_1](#section_4_2)

##################################################################################################################
# 1. Introduction <a class="anchor" id="chapter1"></a>
##################################################################################################################

## 1.1 StatsBomb API Information  <a class="anchor" id="section_1_1"></a>

StatsBomb provides access to the following 7 APIs: 

__1. Competitions__:
- Provides data for the competitions available for the StatsBomb account.
- Each row of data is each season available for each competition.

__2. Matches__:
- Provides data for the matches played in a specific league and season.
- Each row of data is a match played in a specific league and season.

__3. Team Season__:
- Provides aggregated data for teams in a specific league and season.
- Each row of data is a team in a specific league and season.

__4. Player Season__:
- Provides aggregated data for the players in a specific league and season.
- Each row of data is a player in a specific league and season.

__5. Player Match Stats__: 
- Provides aggregated player data for a player in a match.
- Each row of data is the aggregaated statistics for a player that has played in a match.

__6. Events__:
- Provides detailed events data for a match.
- Each row of data is an events that occured in the match.

__7. Lineups__:
- Provides detailed events data for a match.
- Each row of data is an events that occured in the match.

<br>
<center> _______________________________________________________________________________________________ <center> 
<br> 

Some of the APIs require information from the data within the other APIs. For example, the URL for the Matches API requires the user to input a competition_id and season_id. They can only be known by inspecting the competition_id's and season_id's in the data from the Competitions API. 

Below, Fig.1 shows the different APIs (blue boxes), and the solid lines represent the relationships between them:

<img src = "Diagrams/Statsbomb_API_Structure.jpeg" width = "800" align = "center">
<center> <figcaption> Fig.1 </figcaption>  <center>


## 1.2 Notebook Instructions  <a class="anchor" id="section_1_2"></a>   

This notebook allows the user to extract, transform and load (ETL) all five StatsBomb APIs. 
    
Below are the steps to perform the ETL process:
    
    
__1. Inspect Competitions API__: At the beginning of the notebook in section _'2.1 Obtain Competitions Data'_ the user inspects the competitions and seasons available to extract. 

__2. Choose parameters__: In section _'2.2 Choose Competition, Season & Time Range'_, the user chooses the competition, season and date range to extract. Once these have been selected the user can choose whether to extract data from a specific team within the parameters chosen or extract data from all teams.
 
__3. Extract__: Extract data for the Matches, Team Season, Player Season, Player Match, Events and Lineups APIs. Note: if the user has choosen to extract a specific team in section _'2.2 Choose Competition, Season & Time Range'_, the user will be prompted to choose the specific team.
    
__4. Transform__: Transform the seven APIs, and prepare them into a tabular format ready for loading into a relational database. Note: Three additional tables are created in the Transformation stage:

1. Shots Freeze Frame - This table derives from the Events API. The table contains the x,y coordinates of each player on the pitch when a shot event occurs in a match.

2. Formations - This table derives from the Lineups API. The table contains both teams starting formations for each match and any additional formation changes throughout the match and the time the formation change occured. 

3. Team Match Stats - This table derives from the Player Match Stats API. The table aggregates the statistics for each player that played in a match.
    
Fig.1 shows the tables (green boxes) output from the Transformation stage, and the dashed lines show  which API provides the data for them.

__5. Load__: Section 4 allows to user to load two locations:

1. Export to .csv - The user can choose which of the tables output from the Transformation stage they want to export to .csv. When exectued, a specific file path in created in the users root directory where this notebook is stored, and the .csv file is exported.


2. Load to layer_1 - The tables output from the Transformation stage are loaded a database called 'layer_1' which contains no schema. If the table doesn't exist, the tables are created and data inserted. If the tables do exist, any new data will be inserted into tables in the database. Note: If the data already exists in the tables in the database, the new rows of data are ignored. If an existing row of data in the database has new updated information, then the data in the database table is updated.

## 1.3 Load Packages <a class="anchor" id="section_1_3"></a>

In [1]:
# For extraction
import requests
import json

# For transformation
import pandas as pd
import numpy as np
from pandas import json_normalize
from Functions.custom_functions import flatten_json, team_match_calulations
import warnings
warnings.filterwarnings('ignore')

# For load
from datetime import datetime
import os
from sqlalchemy import create_engine, inspect
from pathlib import Path
from Functions.custom_functions import load_to_db, psql_insert_copy

#import mplsoccer.statsbomb
#from mplsoccer.statsbomb import read_event, read_competition, read_match

## 1.4 StatsBomb Credentials <a class="anchor" id="section_1_4"></a>

In [2]:
# Statsbomb credentials
from Credentials.project_credentials import statsbomb_credentials

# Get credentials
creds = statsbomb_credentials()
username = creds["user"]
password = creds["password"]

# Use credentials with requests
auth = requests.auth.HTTPBasicAuth(username, password)

del [creds, username, password]

## 1.5 Optional View of DataFrames <a class="anchor" id="section_1_5"></a>

In [3]:
# view all columns DataFrame
pd.set_option('display.max_columns', None)

# view reduced columns DataFrame
#pd.reset_option(‘max_columns’)

# view all rows DataFrame
#pd.set_option('display.max_rows', None)

# view reduced rows DataFrame
#pd.reset_option(‘max_rows’)

##################################################################################################################
# 2. Extraction<a class="anchor" id="chapter2"></a>
##################################################################################################################

## 2.1 Extract Competitions Data <a class="anchor" id="section_2_1"></a>

In [4]:
# URL
competitions_url = "https://data.statsbombservices.com/api/v4/competitions"

# Create .json object
competitions_data = requests.get(competitions_url, auth = auth).json()

# Create DataFrame
competitions_df = json_normalize(competitions_data)

In [5]:
# View Competitions DataFrame
competitions_df

,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
0,47,235,Austria,Bundesliga,male,False,False,2022/2023,2023-03-27T16:03:45.941942,2023-03-29T03:18:23.660943,2023-03-29T03:18:23.660943,2023-03-27T16:03:45.941942
1,47,108,Austria,Bundesliga,male,False,False,2021/2022,2023-02-24T21:33:32.663870,2022-09-30T08:53:54.151174,2022-09-30T08:53:54.151174,2023-02-24T21:33:32.663870
2,47,90,Austria,Bundesliga,male,False,False,2020/2021,2023-02-24T18:55:18.759573,2021-10-05T19:22:45.888506,None,2023-02-24T18:55:18.759573
3,63,235,Netherlands,Eerste Divisie,male,False,False,2022/2023,2023-03-30T00:53:51.238802,None,None,2023-03-30T00:53:51.238802
4,63,108,Netherlands,Eerste Divisie,male,False,False,2021/2022,2023-01-06T12:32:26.686798,2021-10-28T15:33:03.165385,None,2023-01-06T12:32:26.686798
5,63,90,Netherlands,Eerste Divisie,male,False,False,2020/2021,2023-02-25T14:55:08.501713,2021-06-13T16:17:31.694,None,2023-02-25T14:55:08.501713
6,13,235,Portugal,Liga NOS,male,False,False,2022/2023,2023-03-30T04:41:42.756071,2023-03-25T21:08:29.207381,2023-03-25T21:08:29.207381,2023-03-30T04:41:42.756071
7,13,108,Portugal,Liga NOS,male,False,False,2021/2022,2023-02-27T21:15:30.073150,2022-11-30T19:11:39.243513,2022-11-30T19:11:39.243513,2023-02-27T21:15:30.073150
8,13,90,Portugal,Liga NOS,male,False,False,2020/2021,2022-12-05T14:38:31.465091,2022-08-16T21:28:06.391216,2022-08-16T21:17:28.175124,2022-12-05T14:38:31.465091
9,8,235,France,Ligue 2,male,False,False,2022/2023,2023-03-30T03:28:17.248703,2023-03-22T16:45:28.276005,2023-03-22T16:45:28.276005,2023-03-30T03:28:17.248703


## 2.2 Choose Competition, Season & Time Range <a class="anchor" id="section_2_2"></a>

In [8]:
# Choose Competition
competition_required = input("Enter competition_id: ")

# Choose Season
season_required = input("Enter season_id: ")

# Choose beginning of date range NB includes matches from date chosen
start_date_range = input("Enter start date (format = YYYY-MM-DD): ")

# Choose end of date range NB includes matches from date chosen
end_date_range = input("Enter end date (format = YYYY-MM-DD): ")

# Optional filter for specific team in competition
team_filter = input("Do you want to filter for a specific team? (yes or no): ")


Enter competition_id: 80
Enter season_id: 235
Enter start date (format = YYYY-MM-DD): 2023-03-15
Enter end date (format = YYYY-MM-DD): 2023-03-30
Do you want to filter for a specific team? (yes or no): yes


## 2.3 Extract Matches Data  <a class="anchor" id="section_2_3"></a>

In [9]:
# Create URL
matches_url = f"https://data.statsbombservices.com/api/v5/competitions/{competition_required}/seasons/{season_required}/matches"

# Create .json object for whole season
matches_data_1 = requests.get(matches_url, auth = auth).json()

# Create DataFrame
matches_df = json_normalize(matches_data_1)

# Filter for data range
matches_df = matches_df.loc[matches_df["match_date"].between(start_date_range, end_date_range)]

# Select only available matches NB. future matches included in the API, this excludes them.
matches_df = matches_df.loc[matches_df['match_status'] == "available"]

# If user wants to filter for specific team in competition
if team_filter == 'yes':
    
    # Obtain list of clubs in the matches
    teams_list = pd.unique(matches_df[['home_team.home_team_name', 'away_team.away_team_name']].
                           values.ravel('K')).tolist()

    # Choose team to filter
    team_required = input(f'Which team do you want to filter? \
                          (copy and paste a team name from below): \n \n {teams_list} \n \n')
    
    # Filter DataFrame for chosen team
    matches_df = matches_df.loc[(matches_df['home_team.home_team_name'] == f'{team_required}') |
                                (matches_df['away_team.away_team_name'] == f'{team_required}')]
else:
    pass

# Retrieve available Match id's in a list 
match_ids = matches_df['match_id'].tolist()

# Filter .json object 'matches_data_1' for date range
matches_data_2 = list(filter(lambda x: x['match_id'] in match_ids, matches_data_1))


Which team do you want to filter? (copy and paste a team name from below): 
 
 ['FC Zürich', 'Lugano', 'Sion', 'BSC Young Boys', 'Winterthur', 'Luzern', 'Servette', 'Grasshopper', 'Basel', 'St. Gallen'] 
 
FC Zürich


In [10]:
# View DataFrame
print(matches_df.shape)
matches_df

(1, 48)


,match_id,match_date,kick_off,home_score,away_score,attendance,behind_closed_doors,neutral_ground,play_status,match_status,match_status_360,last_updated,last_updated_360,match_week,competition.competition_id,competition.country_name,competition.competition_name,season.season_id,season.season_name,home_team.home_team_id,home_team.home_team_name,home_team.home_team_gender,home_team.home_team_youth,home_team.home_team_group,home_team.country.id,home_team.country.name,home_team.managers,away_team.away_team_id,away_team.away_team_name,away_team.away_team_gender,away_team.away_team_youth,away_team.away_team_group,away_team.country.id,away_team.country.name,away_team.managers,metadata.data_version,metadata.shot_fidelity_version,metadata.xy_fidelity_version,competition_stage.id,competition_stage.name,stadium.id,stadium.name,stadium.country.id,stadium.country.name,referee.id,referee.name,referee.country.id,referee.country.name
0,3838830,2023-03-18,19:00:00.000,2.0,1.0,None,False,False,Normal,available,available,2023-03-28T09:27:46.840388,2023-03-23T08:43:32.601731,25,80,Switzerland,Super League,235,2022/2023,995,FC Zürich,male,False,None,221,Switzerland,"[{'id': 310, 'name': 'Bo Henriksen', 'nickname...",1148,Luzern,male,False,None,221,Switzerland,"[{'id': 4179, 'name': 'Mario Frick', 'nickname...",1.1.0,2,2,1,Regular Season,527,Stadion Letzigrund,221,Switzerland,1216.0,Urs Schnyder,221.0,Switzerland


## 2.4 Extract Team Season Data <a class="anchor" id="section_2_4"></a>

In [11]:
# URL
team_season_url = f'https://data.statsbombservices.com/api/v2/competitions/{competition_required}/seasons/{season_required}/team-stats'

# Create .json object
team_season_data = requests.get(team_season_url, auth = auth).json()

# Create DataFrame
team_season_df = json_normalize(team_season_data)


In [12]:
# View DataFrame
print(team_season_df.shape)
team_season_df

(10, 181)


,account_id,team_name,team_id,competition_id,competition_name,season_id,season_name,team_female,team_season_matches,team_season_minutes,team_season_gd,team_season_xgd,team_season_np_shots_pg,team_season_op_shots_pg,team_season_op_shots_outside_box_pg,team_season_sp_shots_pg,team_season_np_xg_pg,team_season_op_xg_pg,team_season_sp_xg_pg,team_season_np_xg_per_shot,team_season_np_shot_distance,team_season_op_shot_distance,team_season_sp_shot_distance,team_season_possessions,team_season_possession,team_season_directness,team_season_pace_towards_goal,team_season_gk_pass_distance,team_season_gk_long_pass_ratio,team_season_box_cross_ratio,team_season_passes_inside_box_pg,team_season_defensive_distance,team_season_ppda,team_season_defensive_distance_ppda,team_season_opp_passing_ratio,team_season_opp_final_third_pass_ratio,team_season_np_shots_conceded_pg,team_season_op_shots_conceded_pg,team_season_op_shots_conceded_outside_box_pg,team_season_sp_shots_conceded_pg,team_season_np_xg_conceded_pg,team_season_op_xg_conceded_pg,team_season_sp_xg_conceded_pg,team_season_np_xg_per_shot_conceded,team_season_np_shot_distance_conceded,team_season_op_shot_distance_conceded,team_season_sp_shot_distance_conceded,team_season_deep_completions_conceded_pg,team_season_passes_inside_box_conceded_pg,team_season_corners_pg,team_season_corner_xg_pg,team_season_xg_per_corner,team_season_free_kicks_pg,team_season_free_kick_xg_pg,team_season_xg_per_free_kick,team_season_direct_free_kicks_pg,team_season_direct_free_kick_xg_pg,team_season_xg_per_direct_free_kick,team_season_throw_ins_pg,team_season_throw_in_xg_pg,team_season_xg_per_throw_in,team_season_ball_in_play_time,team_season_counter_attacking_shots_pg,team_season_high_press_shots_pg,team_season_shots_in_clear_pg,team_season_counter_attacking_shots_conceded_pg,team_season_shots_in_clear_conceded_pg,team_season_aggressive_actions_pg,team_season_aggression,team_season_goals_pg,team_season_own_goals_pg,team_season_penalty_goals_pg,team_season_goals_conceded_pg,team_season_opposition_own_goals_pg,team_season_penalty_goals_conceded_pg,team_season_shots_from_corners_pg,team_season_goals_from_corners_pg,team_season_shots_from_free_kicks_pg,team_season_goals_from_free_kicks_pg,team_season_direct_free_kick_goals_pg,team_season_shots_from_direct_free_kicks_pg,team_season_shots_from_throw_ins_pg,team_season_goals_from_throw_ins_pg,team_season_direct_free_kick_goals_conceded_pg,team_season_shots_from_direct_free_kicks_conceded_pg,team_season_corners_conceded_pg,team_season_corner_xg_conceded_pg,team_season_shots_from_corners_conceded_pg,team_season_goals_from_corners_conceded_pg,team_season_free_kicks_conceded_pg,team_season_free_kick_xg_conceded_pg,team_season_shots_from_free_kicks_conceded_pg,team_season_goals_from_free_kicks_conceded_pg,team_season_direct_free_kicks_conceded_pg,team_season_direct_free_kick_xg_conceded_pg,team_season_throw_ins_conceded_pg,team_season_throw_in_xg_conceded_pg,team_season_shots_from_throw_ins_conceded_pg,team_season_goals_from_throw_ins_conceded_pg,team_season_corner_shot_ratio,team_season_corner_goal_ratio,team_season_free_kick_shot_ratio,team_season_free_kick_goal_ratio,team_season_direct_free_kick_goal_ratio,team_season_throw_in_shot_ratio,team_season_throw_in_goal_ratio,team_season_xg_per_corner_conceded,team_season_corner_shot_ratio_conceded,team_season_corner_goal_ratio_conceded,team_season_xg_per_free_kick_conceded,team_season_free_kick_shot_ratio_conceded,team_season_free_kick_goal_ratio_conceded,team_season_xg_per_direct_free_kick_conceded,team_season_direct_free_kick_goal_ratio_conceded,team_season_xg_per_throw_in_conceded,team_season_throw_in_shot_ratio_conceded,team_season_throw_in_goal_ratio_conceded,team_season_direct_free_kick_shot_ratio,team_season_direct_free_kick_shot_ratio_conceded,team_season_sp_pg,team_season_xg_per_sp,team_season_sp_shot_ratio,team_season_sp_goals_pg,team_season_sp_goal_ratio,team_season_sp_pg_conceded,team_season_xg_per_sp_conceded,team_season_s

## 2.5 Extract Player Season Data <a class="anchor" id="section_2_5"></a>

In [13]:
# URL
player_season_url = f'https://data.statsbombservices.com/api/v2/competitions/{competition_required}/seasons/{season_required}/player-stats'

# Create .json object
player_season_data = requests.get(player_season_url, auth = auth).json()

# Create DataFrame
player_season_df = json_normalize(player_season_data)


In [14]:
# View DataFrame
print(player_season_df.shape)
player_season_df

(283, 168)


,account_id,player_id,player_name,team_id,team_name,competition_id,competition_name,season_id,season_name,country_id,birth_date,player_female,player_first_name,player_last_name,player_known_name,player_weight,player_height,player_season_minutes,player_season_np_xg_per_shot,player_season_np_xg_90,player_season_np_shots_90,player_season_goals_90,player_season_npga_90,player_season_xa_90,player_season_key_passes_90,player_season_op_key_passes_90,player_season_assists_90,player_season_through_balls_90,player_season_passes_into_box_90,player_season_op_passes_into_box_90,player_season_touches_inside_box_90,player_season_op_passes_into_and_touches_inside_box_90,player_season_tackles_90,player_season_interceptions_90,player_season_tackles_and_interceptions_90,player_season_padj_tackles_90,player_season_padj_interceptions_90,player_season_padj_tackles_and_interceptions_90,player_season_challenge_ratio,player_season_dribbles_90,player_season_fouls_90,player_season_dribbled_past_90,player_season_dispossessions_90,player_season_long_ball_ratio,player_season_long_balls_90,player_season_blocks_per_shot,player_season_clearance_90,player_season_aerial_ratio,player_season_aerial_wins_90,player_season_op_passes_90,player_season_forward_pass_proportion,player_season_backward_pass_proportion,player_season_sideways_pass_proportion,player_season_op_f3_passes_90,player_season_op_f3_forward_pass_proportion,player_season_op_f3_backward_pass_proportion,player_season_op_f3_sideways_pass_proportion,player_season_shot_on_target_ratio,player_season_passing_ratio,player_season_conversion_ratio,player_season_npg_90,player_season_crosses_90,player_season_crossing_ratio,player_season_penalty_wins_90,player_season_padj_clearances_90,player_season_passes_inside_box_90,player_season_xgchain_90,player_season_op_xgchain_90,player_season_xgbuildup_90,player_season_op_xgbuildup_90,player_season_op_xa_90,player_season_op_assists_90,player_season_xgchain,player_season_op_xgchain,player_season_xgbuildup,player_season_op_xgbuildup,player_season_xgchain_per_possession,player_season_op_xgchain_per_possession,player_season_xgbuildup_per_possession,player_season_op_xgbuildup_per_possession,primary_position,secondary_position,player_season_pressured_long_balls_90,player_season_unpressured_long_balls_90,player_season_aggressive_actions_90,player_season_pressures_90,player_season_turnovers_90,player_season_pressure_regains_90,player_season_box_cross_ratio,player_season_sp_xa_90,player_season_shot_touch_ratio,player_season_deep_progressions_90,player_season_fouls_won_90,player_season_shots_faced_90,player_season_goals_faced_90,player_season_np_xg_faced_90,player_season_np_psxg_faced_90,player_season_save_ratio,player_season_xs_ratio,player_season_gsaa_90,player_season_gsaa_ratio,player_season_left_foot_ratio,player_season_pressured_passing_ratio,player_season_passes_pressed_ratio,player_season_pass_length,player_season_s_pass_length,player_season_p_pass_length,player_season_ps_pass_length,player_season_ot_shots_faced_90,player_season_npot_psxg_faced_90,player_season_ot_shots_faced_ratio,player_season_np_optimal_gk_dlength,player_season_clcaa,player_season_pass_into_pressure_ratio,player_season_pass_into_danger_ratio,player_season_da_aggressive_distance,player_season_positive_outcome_score,player_season_appearances,player_season_average_minutes,player_season_positive_outcome_90,player_season_sp_assists_90,player_season_sp_key_passes_90,player_season_npxgxa_90,player_season_over_under_performance_90,player_season_shots_key_passes_90,player_season_failed_dribbles_90,player_season_total_dribbles_90,player_season_dribble_ratio,player_season_sp_passes_into_box_90,player_season_penalty_conversion_ratio,player_season_pass_length_ratio,player_season_pressured_pass_length_ratio,player_season_change_in_passing_ratio,player_season_pressured_change_in_pass_length,player_season_carries_90,player_season_carry_ratio,player_season_carry_length,player_season_dribble_faced_ratio,player_season_coun

## 2.6 Extract Player Match Data <a class="anchor" id="section_2_6"></a>

In [15]:
# Empty dataframe
player_match_df = pd.DataFrame()

# Counter to confirm all data from Match id's successfully retrieved
counter = 0

# Loop through match_ids
for i in match_ids:

    # Create URL
    player_match_url = f"https://data.statsbombservices.com/api/v2/matches/{i}/player-stats"

    # Create .json object
    player_match_data = requests.get(player_match_url, auth = auth).json()

    # Create temporary DataFrame
    player_match_df_temp = json_normalize(player_match_data)

    # Append temporary DataFrame
    player_match_df = pd.concat([player_match_df, player_match_df_temp])

    # Counter plus 1
    counter += 1
    if (counter % 20) == 0:
        print("Data from " + str(counter) + " match_ids retrieved")
        
print("-" * 40)

# Confirm Match id's data succesfully retreived 
print("Available Match id's = " + str(len(match_ids)))
print("Match id's succesfully retrieved = " + str(counter)) 

del [player_match_df_temp, counter]

----------------------------------------
Available Match id's = 1
Match id's succesfully retrieved = 1


In [16]:
# View DataFrame
print(player_match_df.shape)
#player_match_df

(28, 97)


## 2.7 Extract Events Data <a class="anchor" id="section_2_7"></a>

In [17]:
# Empty DataFrame 
events_df = pd.DataFrame()

# Counter to confirm all data from Match id's successfully retrieved
counter = 0

# Loop through match_ids
for i in match_ids:

    # Create URL
    events_url = f"https://data.statsbombservices.com/api/v6/events/{i}"

    # Create .json object
    events_data = requests.get(events_url, auth = auth).json()

    # Create temporary DataFrame
    events_df_temp = json_normalize(events_data)
    
    # Add match_id to column at index 0
    events_df_temp.insert(0, "match_id", i, True)

    # Append temporary DataFrame
    events_df = pd.concat([events_df, events_df_temp])

    # Counter plus 1
    counter += 1
    if (counter % 20) == 0:
        print("Data from " + str(counter) + " match_ids retrieved")

print("-" * 40)

# Confirm Match id's data succesfully retreived 
print("Available Match id's = " + str(len(match_ids)))
print("Match id's succesfully retrieved = " + str(counter))

del [events_df_temp, counter]

----------------------------------------
Available Match id's = 1
Match id's succesfully retrieved = 1


In [18]:
# View DataFrame
print(events_df.shape)
#events_df

(3521, 128)


## 2.8 Extract Lineups Data <a class="anchor" id="section_2_8"></a>

In [19]:
# Empty DataFrame
lineups_df = pd.DataFrame()

# Counter to confirm all data from Match id's successfully retrieved
counter = 0

# Loop through match_ids
for i in match_ids:

    # Create URL
    lineups_url = f"https://data.statsbombservices.com/api/v4/lineups/{i}"

    # Create .json object
    lineups_data = requests.get(lineups_url, auth = auth).json()

    # Create temporary DataFrame
    lineups_df_temp = json_normalize(lineups_data)
    
    # Add match_id to column at index 0
    lineups_df_temp.insert(0, "match_id", i, True)

    # Append temporary DataFrame
    lineups_df = pd.concat([lineups_df, lineups_df_temp])

    # Counter plus 1
    counter += 1
    if (counter % 20) == 0:
        print("Data from " + str(counter) + " match_ids retrieved")

print("-" * 40)

# Confirm Match id's data succesfully retreived 
print("Available Match id's = " + str(len(match_ids)))
print("Match id's succesfully retrieved = " + str(counter))

del [lineups_df_temp, counter]

----------------------------------------
Available Match id's = 1
Match id's succesfully retrieved = 1


In [20]:
# View DataFrame
print(lineups_df.shape)
#lineups_df

(2, 6)


##################################################################################################################
# 3. Transformation <a class="anchor" id="chapter3"></a>
##################################################################################################################

## 3.1 Transform Competitions Data <a class="anchor" id="section_3_1"></a>

In [21]:
# Save new instance
competitions_df_2 = competitions_df

# Assign an id column for table
competitions_df_2.insert(0, 'comp_season_id', competitions_df_2.competition_id.astype(str) +
                                              competitions_df_2.season_id.astype(str))

# Convert columns to int type
competitions_df_2 = competitions_df_2.astype({'comp_season_id':'int64'})

# Convert columns to datetime type
cols = ['match_updated','match_updated_360','match_available_360','match_available']
competitions_df_2[cols] = competitions_df_2[cols].astype('datetime64[ns]')

# Rename Columns
competitions_df_2.rename(columns = {'country_name':'competition_country_name'}, inplace = True)

# Assign a name for DataFrame
competitions_df_2.name = "competitions"

del cols

In [22]:
# View DataFrame
print(competitions_df_2.shape)
#competitions_df_2

(29, 13)


## 3.2 Transform Matches Data <a class="anchor" id="section_3_2"></a>

In [23]:
# Create DataFrame
matches_df_2 = pd.DataFrame([flatten_json(x) for x in matches_data_2])

# Convert columns to datetime type
cols = ['match_date','home_team_managers_0_dob', 'away_team_managers_0_dob']
matches_df_2[cols] = matches_df_2[cols].astype('datetime64[ns]')

# Columns to rename
col_dict = {'competition_competition_id': 'competition_id',
            'competition_competition_name': 'competition_name',
            'season_season_id': 'season_id',
            'season_season_name': 'season_name',
            'home_team_home_team_id': 'home_team_id',
            'home_team_home_team_name': 'home_team_name',
            'home_team_home_team_gender': 'home_team_gender',
            'home_team_home_team_youth': 'home_team_youth',
            'home_team_home_team_group': 'home_team_group',
            'home_team_managers_0_id': 'home_team_managers_id',
            'home_team_managers_0_name': 'home_team_managers_name',
            'home_team_managers_0_nickname': 'home_team_managers_nickname',
            'home_team_managers_0_dob': 'home_team_managers_dob',
            'home_team_managers_0_country_id': 'home_team_managers_country_id',
            'home_team_managers_0_country_name': 'home_team_managers_country_name',
            'away_team_away_team_id': 'away_team_id',
            'away_team_away_team_name': 'away_team_name',
            'away_team_away_team_gender': 'away_team_gender',
            'away_team_away_team_youth': 'away_team_youth',
            'away_team_away_team_group': 'away_team_group',
            'away_team_managers_0_id': 'away_team_managers_id',
            'away_team_managers_0_name': 'away_team_managers_name',
            'away_team_managers_0_nickname': 'away_team_managers_nickname',
            'away_team_managers_0_dob': 'away_team_managers_dob',
            'away_team_managers_0_country_id': 'away_team_managers_country_id',
            'away_team_managers_0_country_name': 'away_team_managers_country_name',
            }

# Rename columns
matches_df_2.rename(columns = col_dict, inplace = True)

# Assign a name for DataFrame
matches_df_2.name = "matches"

del [cols, col_dict]

In [24]:
# View DataFrame
print(matches_df_2.shape)
matches_df_2

(1, 58)


,match_id,match_date,kick_off,competition_id,competition_country_name,competition_name,season_id,season_name,home_team_id,home_team_name,home_team_gender,home_team_youth,home_team_group,home_team_country_id,home_team_country_name,home_team_managers_id,home_team_managers_name,home_team_managers_nickname,home_team_managers_dob,home_team_managers_country_id,home_team_managers_country_name,away_team_id,away_team_name,away_team_gender,away_team_youth,away_team_group,away_team_country_id,away_team_country_name,away_team_managers_id,away_team_managers_name,away_team_managers_nickname,away_team_managers_dob,away_team_managers_country_id,away_team_managers_country_name,home_score,away_score,attendance,behind_closed_doors,neutral_ground,play_status,match_status,match_status_360,last_updated,last_updated_360,metadata_data_version,metadata_shot_fidelity_version,metadata_xy_fidelity_version,match_week,competition_stage_id,competition_stage_name,stadium_id,stadium_name,stadium_country_id,stadium_country_name,referee_id,referee_name,referee_country_id,referee_country_name
0,3838830,2023-03-18,19:00:00.000,80,Switzerland,Super League,235,2022/2023,995,FC Zürich,male,False,None,221,Switzerland,310,Bo Henriksen,None,1975-02-07,61,Denmark,1148,Luzern,male,False,None,221,Switzerland,4179,Mario Frick,None,1974-09-07,131,Liechtenstein,2,1,None,False,False,Normal,available,available,2023-03-28T09:27:46.840388,2023-03-23T08:43:32.601731,1.1.0,2,2,25,1,Regular Season,527,Stadion Letzigrund,221,Switzerland,1216,Urs Schnyder,221,Switzerland


## 3.3 Transform Team Season Data <a class="anchor" id="section_3_3"></a>

In [25]:
# Save new instance
team_season_df_2 = team_season_df

# Create a unique id column
team_season_df_2.insert(0, 'team_season_id', team_season_df_2.competition_id.astype(str) +
                                             team_season_df_2.season_id.astype(str) +
                                             team_season_df_2.team_id.astype(str))

# Convert columns to int type
team_season_df_2 = team_season_df_2.astype({'team_season_id':'int64'})

# Assign a name for DataFrame
team_season_df_2.name = "team_season"


In [26]:
# View DataFrame
print(team_season_df_2.shape)
team_season_df_2

(10, 182)


,team_season_id,account_id,team_name,team_id,competition_id,competition_name,season_id,season_name,team_female,team_season_matches,team_season_minutes,team_season_gd,team_season_xgd,team_season_np_shots_pg,team_season_op_shots_pg,team_season_op_shots_outside_box_pg,team_season_sp_shots_pg,team_season_np_xg_pg,team_season_op_xg_pg,team_season_sp_xg_pg,team_season_np_xg_per_shot,team_season_np_shot_distance,team_season_op_shot_distance,team_season_sp_shot_distance,team_season_possessions,team_season_possession,team_season_directness,team_season_pace_towards_goal,team_season_gk_pass_distance,team_season_gk_long_pass_ratio,team_season_box_cross_ratio,team_season_passes_inside_box_pg,team_season_defensive_distance,team_season_ppda,team_season_defensive_distance_ppda,team_season_opp_passing_ratio,team_season_opp_final_third_pass_ratio,team_season_np_shots_conceded_pg,team_season_op_shots_conceded_pg,team_season_op_shots_conceded_outside_box_pg,team_season_sp_shots_conceded_pg,team_season_np_xg_conceded_pg,team_season_op_xg_conceded_pg,team_season_sp_xg_conceded_pg,team_season_np_xg_per_shot_conceded,team_season_np_shot_distance_conceded,team_season_op_shot_distance_conceded,team_season_sp_shot_distance_conceded,team_season_deep_completions_conceded_pg,team_season_passes_inside_box_conceded_pg,team_season_corners_pg,team_season_corner_xg_pg,team_season_xg_per_corner,team_season_free_kicks_pg,team_season_free_kick_xg_pg,team_season_xg_per_free_kick,team_season_direct_free_kicks_pg,team_season_direct_free_kick_xg_pg,team_season_xg_per_direct_free_kick,team_season_throw_ins_pg,team_season_throw_in_xg_pg,team_season_xg_per_throw_in,team_season_ball_in_play_time,team_season_counter_attacking_shots_pg,team_season_high_press_shots_pg,team_season_shots_in_clear_pg,team_season_counter_attacking_shots_conceded_pg,team_season_shots_in_clear_conceded_pg,team_season_aggressive_actions_pg,team_season_aggression,team_season_goals_pg,team_season_own_goals_pg,team_season_penalty_goals_pg,team_season_goals_conceded_pg,team_season_opposition_own_goals_pg,team_season_penalty_goals_conceded_pg,team_season_shots_from_corners_pg,team_season_goals_from_corners_pg,team_season_shots_from_free_kicks_pg,team_season_goals_from_free_kicks_pg,team_season_direct_free_kick_goals_pg,team_season_shots_from_direct_free_kicks_pg,team_season_shots_from_throw_ins_pg,team_season_goals_from_throw_ins_pg,team_season_direct_free_kick_goals_conceded_pg,team_season_shots_from_direct_free_kicks_conceded_pg,team_season_corners_conceded_pg,team_season_corner_xg_conceded_pg,team_season_shots_from_corners_conceded_pg,team_season_goals_from_corners_conceded_pg,team_season_free_kicks_conceded_pg,team_season_free_kick_xg_conceded_pg,team_season_shots_from_free_kicks_conceded_pg,team_season_goals_from_free_kicks_conceded_pg,team_season_direct_free_kicks_conceded_pg,team_season_direct_free_kick_xg_conceded_pg,team_season_throw_ins_conceded_pg,team_season_throw_in_xg_conceded_pg,team_season_shots_from_throw_ins_conceded_pg,team_season_goals_from_throw_ins_conceded_pg,team_season_corner_shot_ratio,team_season_corner_goal_ratio,team_season_free_kick_shot_ratio,team_season_free_kick_goal_ratio,team_season_direct_free_kick_goal_ratio,team_season_throw_in_shot_ratio,team_season_throw_in_goal_ratio,team_season_xg_per_corner_conceded,team_season_corner_shot_ratio_conceded,team_season_corner_goal_ratio_conceded,team_season_xg_per_free_kick_conceded,team_season_free_kick_shot_ratio_conceded,team_season_free_kick_goal_ratio_conceded,team_season_xg_per_direct_free_kick_conceded,team_season_direct_free_kick_goal_ratio_conceded,team_season_xg_per_throw_in_conceded,team_season_throw_in_shot_ratio_conceded,team_season_throw_in_goal_ratio_conceded,team_season_direct_free_kick_shot_ratio,team_season_direct_free_kick_shot_ratio_conceded,team_season_sp_pg,team_season_xg_per_sp,team_season_sp_shot_ratio,team_season_sp_goals_pg,team_season_sp_goal_ratio,team_season_sp_pg_conceded,team_season_xg_per_sp_concede

## 3.4 Transform Player Season Data <a class="anchor" id="section_3_4"></a>

In [27]:
# Save new instance
player_season_df_2 = player_season_df

# Create a unique id column
player_season_df_2.insert(0, 'player_season_id', player_season_df_2.competition_id.astype(str) +
                                                 player_season_df_2.season_id.astype(str) + 
                                                 player_season_df_2.player_id.astype(str) +
                                                 player_season_df_2.team_id.astype(str))

# Convert columns to int type
player_season_df_2 = player_season_df_2.astype({'player_season_id':'int64'})

# Assign a name for DataFrame
player_season_df_2.name = "player_season"


In [29]:
# View DataFrame
print(player_season_df_2.shape)
player_season_df_2

(283, 169)


,player_season_id,account_id,player_id,player_name,team_id,team_name,competition_id,competition_name,season_id,season_name,country_id,birth_date,player_female,player_first_name,player_last_name,player_known_name,player_weight,player_height,player_season_minutes,player_season_np_xg_per_shot,player_season_np_xg_90,player_season_np_shots_90,player_season_goals_90,player_season_npga_90,player_season_xa_90,player_season_key_passes_90,player_season_op_key_passes_90,player_season_assists_90,player_season_through_balls_90,player_season_passes_into_box_90,player_season_op_passes_into_box_90,player_season_touches_inside_box_90,player_season_op_passes_into_and_touches_inside_box_90,player_season_tackles_90,player_season_interceptions_90,player_season_tackles_and_interceptions_90,player_season_padj_tackles_90,player_season_padj_interceptions_90,player_season_padj_tackles_and_interceptions_90,player_season_challenge_ratio,player_season_dribbles_90,player_season_fouls_90,player_season_dribbled_past_90,player_season_dispossessions_90,player_season_long_ball_ratio,player_season_long_balls_90,player_season_blocks_per_shot,player_season_clearance_90,player_season_aerial_ratio,player_season_aerial_wins_90,player_season_op_passes_90,player_season_forward_pass_proportion,player_season_backward_pass_proportion,player_season_sideways_pass_proportion,player_season_op_f3_passes_90,player_season_op_f3_forward_pass_proportion,player_season_op_f3_backward_pass_proportion,player_season_op_f3_sideways_pass_proportion,player_season_shot_on_target_ratio,player_season_passing_ratio,player_season_conversion_ratio,player_season_npg_90,player_season_crosses_90,player_season_crossing_ratio,player_season_penalty_wins_90,player_season_padj_clearances_90,player_season_passes_inside_box_90,player_season_xgchain_90,player_season_op_xgchain_90,player_season_xgbuildup_90,player_season_op_xgbuildup_90,player_season_op_xa_90,player_season_op_assists_90,player_season_xgchain,player_season_op_xgchain,player_season_xgbuildup,player_season_op_xgbuildup,player_season_xgchain_per_possession,player_season_op_xgchain_per_possession,player_season_xgbuildup_per_possession,player_season_op_xgbuildup_per_possession,primary_position,secondary_position,player_season_pressured_long_balls_90,player_season_unpressured_long_balls_90,player_season_aggressive_actions_90,player_season_pressures_90,player_season_turnovers_90,player_season_pressure_regains_90,player_season_box_cross_ratio,player_season_sp_xa_90,player_season_shot_touch_ratio,player_season_deep_progressions_90,player_season_fouls_won_90,player_season_shots_faced_90,player_season_goals_faced_90,player_season_np_xg_faced_90,player_season_np_psxg_faced_90,player_season_save_ratio,player_season_xs_ratio,player_season_gsaa_90,player_season_gsaa_ratio,player_season_left_foot_ratio,player_season_pressured_passing_ratio,player_season_passes_pressed_ratio,player_season_pass_length,player_season_s_pass_length,player_season_p_pass_length,player_season_ps_pass_length,player_season_ot_shots_faced_90,player_season_npot_psxg_faced_90,player_season_ot_shots_faced_ratio,player_season_np_optimal_gk_dlength,player_season_clcaa,player_season_pass_into_pressure_ratio,player_season_pass_into_danger_ratio,player_season_da_aggressive_distance,player_season_positive_outcome_score,player_season_appearances,player_season_average_minutes,player_season_positive_outcome_90,player_season_sp_assists_90,player_season_sp_key_passes_90,player_season_npxgxa_90,player_season_over_under_performance_90,player_season_shots_key_passes_90,player_season_failed_dribbles_90,player_season_total_dribbles_90,player_season_dribble_ratio,player_season_sp_passes_into_box_90,player_season_penalty_conversion_ratio,player_season_pass_length_ratio,player_season_pressured_pass_length_ratio,player_season_change_in_passing_ratio,player_season_pressured_change_in_pass_length,player_season_carries_90,player_season_carry_ratio,player_season_carry_length,player_season_dribble_faced_ratio,p

## 3.5 Transform Player Match Data <a class="anchor" id="section_3_5"></a>

In [30]:
# Save new instance
player_match_df_2 = player_match_df

# Create a unique id column
player_match_df_2.insert(0, 'player_match_id', player_match_df_2.match_id.astype(str) +
                                               player_match_df_2.player_id.astype(str))

# Convert columns to int type
player_match_df_2 = player_match_df_2.astype({'player_match_id':'int64'})

# Assign a name for DataFrame
player_match_df_2.name = "player_match"

In [34]:
# View DataFrame
print(player_match_df_2.shape)
player_match_df_2

(28, 98)


,player_match_id,account_id,match_id,team_id,team_name,player_id,player_name,player_match_minutes,player_match_np_xg_per_shot,player_match_np_xg,player_match_np_shots,player_match_goals,player_match_np_goals,player_match_xa,player_match_key_passes,player_match_op_key_passes,player_match_assists,player_match_through_balls,player_match_passes_into_box,player_match_op_passes_into_box,player_match_touches_inside_box,player_match_tackles,player_match_interceptions,player_match_possession,player_match_dribbled_past,player_match_dribbles_faced,player_match_dribbles,player_match_challenge_ratio,player_match_fouls,player_match_dispossessions,player_match_long_balls,player_match_successful_long_balls,player_match_long_ball_ratio,player_match_shots_blocked,player_match_clearances,player_match_aerials,player_match_successful_aerials,player_match_aerial_ratio,player_match_passes,player_match_successful_passes,player_match_passing_ratio,player_match_op_passes,player_match_forward_passes,player_match_backward_passes,player_match_sideways_passes,player_match_op_f3_passes,player_match_op_f3_forward_passes,player_match_op_f3_backward_passes,player_match_op_f3_sideways_passes,player_match_np_shots_on_target,player_match_crosses,player_match_successful_crosses,player_match_crossing_ratio,player_match_penalties_won,player_match_passes_inside_box,player_match_op_xa,player_match_op_assists,player_match_pressured_long_balls,player_match_unpressured_long_balls,player_match_aggressive_actions,player_match_turnovers,player_match_crosses_into_box,player_match_sp_xa,player_match_op_shots,player_match_touches,player_match_pressure_regains,player_match_box_cross_ratio,player_match_deep_progressions,player_match_shot_touch_ratio,player_match_fouls_won,player_match_xgchain,player_match_op_xgchain,player_match_xgbuildup,player_match_op_xgbuildup,player_match_xgchain_per_possession,player_match_op_xgchain_per_possession,player_match_xgbuildup_per_possession,player_match_op_xgbuildup_per_possession,player_match_pressures,player_match_pressure_duration_total,player_match_pressure_duration_avg,player_match_pressured_action_fails,player_match_counterpressures,player_match_counterpressure_duration_total,player_match_counterpressure_duration_avg,player_match_counterpressured_action_fails,player_match_obv,player_match_obv_pass,player_match_obv_shot,player_match_obv_defensive_action,player_match_obv_dribble_carry,player_match_obv_gk,player_match_deep_completions,player_match_ball_recoveries,player_match_np_psxg,player_match_penalties_faced,player_match_penalties_conceded,player_match_fhalf_ball_recoveries
0,38388305554,437,3838830,995,FC Zürich,5554,Blerim Džemaili,82.566666,0.011305,0.011305,1,0,0,0.322902,3,3,0,0,5,5,2,0,0,0.522044,0,0,1,NaN,1,1,2,1,0.500000,1,1,1,0,0.000000,48,35,0.729167,41,11,11,26,15,4,3,8,1,1,1,1.000000,0,0,0.322902,0,1,2,7,4,1,0.000000,1,85,1,0.200000,3,0.011765,3,0.479776,0.479776,0.202267,0.202267,0.009226,0.009226,0.003890,0.003890,15.0,10.482580,0.698839,3.0,4.0,3.095181,0.773795,0.0,0.947943,0.749793,-0.028440,0.119136,0.107454,NaN,0,10,0.003202,0,0,7
1,38388308508,437,3838830,1148,Luzern,8508,Max Meyer,94.533330,0.085371,0.085371,1,1,0,0.293176,2,0,0,0,4,1,6,3,0,0.477956,3,3,0,0.500000,1,0,7,4,0.571429,1,0,0,0,NaN,57,45,0.789474,51,4,6,47,14,2,2,10,1,1,0,0.000000,0,0,0.000000,0,1,6,15,1,0,0.293176,1,99,5,0.000000,4,0.010101,4,1.267219,0.166385,0.024158,0.000000,0.024847,0.003262,0.000474,0.000000,18.0,20.466711,1.137039,1.0,5.0,2.657924,0.531585,0.0,0.785567,0.195638,0.515074,-0.014380,0.089236,NaN,1,10,0.519781,0,0,6
2,383883029043,437,3838830,995,FC Zürich,29043,Marc Hornschuh,11.966666,NaN,0.000000,0,0,0,0.000000,0,0,0,0,0,0,1,1,1,0.522044,0,0,0,1.000000,0,0,0,0,NaN,0,0,2,1,0.500000,5,3,0.600000,5,1,1,3,0,0,0,0,0,0,0,NaN,0,0,0.000000,0,0,0,2,0,0,0.000000,0,9,1,NaN,0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.922510,0.922510,0.0,0.0,NaN,NaN,0.0,0.001713,-0.000885,NaN,0.002623,-0.000025,NaN,0

### 3.5.1 Team Match Data <a class="anchor" id="section_3_5_1"></a>

In [32]:
# Load calculation rules for each column
team_match_dict = team_match_calulations()

# Group player match statistics
team_match_df = player_match_df_2.groupby(['match_id','team_id', 'team_name']).aggregate(team_match_dict).reset_index()

# Save new instance
team_match_df_2 = team_match_df

# Convert columns to int type
team_match_df_2 = team_match_df_2.astype({'match_id':'int64'})

# Create a unique id column
team_match_df_2.insert(0, 'team_match_id', team_match_df_2.match_id.astype(str) +
                                           team_match_df_2.team_id.astype(str))

# Convert columns to int type
team_match_df_2 = team_match_df_2.astype({'team_match_id':'int64'})

# Assign a name for DataFrame
team_match_df_2.name = "team_match"

del team_match_dict


In [33]:
# View DataFrame
print(team_match_df_2.shape)
team_match_df_2

(2, 95)


,team_match_id,match_id,team_id,team_name,player_match_minutes,player_match_np_xg_per_shot,player_match_np_xg,player_match_np_shots,player_match_goals,player_match_np_goals,player_match_xa,player_match_key_passes,player_match_op_key_passes,player_match_assists,player_match_through_balls,player_match_passes_into_box,player_match_op_passes_into_box,player_match_touches_inside_box,player_match_tackles,player_match_interceptions,player_match_possession,player_match_dribbled_past,player_match_dribbles_faced,player_match_dribbles,player_match_challenge_ratio,player_match_fouls,player_match_dispossessions,player_match_long_balls,player_match_successful_long_balls,player_match_long_ball_ratio,player_match_shots_blocked,player_match_clearances,player_match_aerials,player_match_successful_aerials,player_match_aerial_ratio,player_match_passes,player_match_successful_passes,player_match_passing_ratio,player_match_op_passes,player_match_forward_passes,player_match_backward_passes,player_match_sideways_passes,player_match_op_f3_passes,player_match_op_f3_forward_passes,player_match_op_f3_backward_passes,player_match_op_f3_sideways_passes,player_match_np_shots_on_target,player_match_crosses,player_match_successful_crosses,player_match_crossing_ratio,player_match_penalties_won,player_match_passes_inside_box,player_match_op_xa,player_match_op_assists,player_match_pressured_long_balls,player_match_unpressured_long_balls,player_match_aggressive_actions,player_match_turnovers,player_match_crosses_into_box,player_match_sp_xa,player_match_op_shots,player_match_touches,player_match_pressure_regains,player_match_box_cross_ratio,player_match_deep_progressions,player_match_shot_touch_ratio,player_match_fouls_won,player_match_xgchain,player_match_op_xgchain,player_match_xgbuildup,player_match_op_xgbuildup,player_match_xgchain_per_possession,player_match_op_xgchain_per_possession,player_match_xgbuildup_per_possession,player_match_op_xgbuildup_per_possession,player_match_pressures,player_match_pressure_duration_total,player_match_pressure_duration_avg,player_match_pressured_action_fails,player_match_counterpressures,player_match_counterpressure_duration_total,player_match_counterpressure_duration_avg,player_match_counterpressured_action_fails,player_match_obv,player_match_obv_pass,player_match_obv_shot,player_match_obv_defensive_action,player_match_obv_dribble_carry,player_match_obv_gk,player_match_deep_completions,player_match_ball_recoveries,player_match_np_psxg,player_match_penalties_faced,player_match_penalties_conceded,player_match_fhalf_ball_recoveries
0,3838830995,3838830,995,FC Zürich,1039.866635,0.756130,1.387769,13,2,1,1.269213,9,8,1,0,13,12,28,9,16,0.522044,1,2,12,0.857143,10,15,74,39,0.547698,4,13,41,20,0.384028,521,407,0.768374,470,124,51,346,98,17,11,70,3,13,4,0.342857,0,2,1.207017,1,22,65,86,21,2,0.062196,12,912,28,0.088889,46,0.012321,15,9.762821,8.917125,7.654693,7.654693,0.281916,0.250275,0.226260,0.226260,167.0,154.143712,0.923677,27.0,49.0,50.286873,1.060944,8.0,2.480361,1.876570,-0.059870,-0.248034,0.638380,0.273316,2,115,0.973367,1,1,41
1,38388301148,3838830,1148,Luzern,1040.333302,0.512538,0.785594,8,1,0,0.733808,7,5,0,2,12,7,30,19,18,0.477956,13,17,1,0.630159,16,8,76,45,0.671246,3,12,41,21,0.532468,477,359,0.754936,423,137,39,301,80,23,9,48,2,8,3,0.233333,1,3,0.440632,0,21,67,106,22,2,0.293176,4,808,30,0.100000,37,0.009092,10,3.821891,2.669271,1.955217,1.879272,0.096911,0.074597,0.056115,0.054912,160.0,163.674046,1.068142,30.0,40.0,33.651235,0.876483,4.0,1.920862,0.732315,0.221854,-0.085445,1.253872,-0.201734,5,98,0.615195,1,1,31


## 3.6 Transform Events Data <a class="anchor" id="section_3_6"></a>

### 3.6.1 Events <a class="anchor" id="section_3_6_1"></a>

In [35]:
# Save new instance
events_df_2 = events_df

# Rename column
events_df_2.rename(columns = {'id':'event_id'}, inplace = True)

# Move column
events_df_2 = events_df_2.reindex(columns=["event_id"] + 
                                  [col for col in events_df_2.columns if col != "event_id"])

# Replace '.' from column names with '_'
events_df_2.columns = events_df_2.columns.str.replace('.','_',  regex = True)

# Remove lineups column NB. this data is included in the Lineups API
events_df_2.drop('tactics_lineup', inplace = True, axis = 1)

# Create DataFrame for shot_freeze_frame
shot_freeze_frame_df = events_df_2[['event_id','match_id','shot_freeze_frame']]

# Remove shot_freeze_frame
events_df_2.drop('shot_freeze_frame', inplace = True, axis = 1)

# Save copy of events_df_2
df_types = events_df_2

# Remove square brackets
events_df_2 = events_df_2.astype(str).replace({"\[":"", "\]":""}, regex = True)

# Assign original dtypes of events_df_2
events_df_2 = events_df_2.astype(df_types.dtypes.to_dict())

# Replace 'nan' string with NAN value
events_df_2 = events_df_2.replace('nan', np.nan)

# Split location values into new columns
events_df_2[['location_x', 'location_y', 'location_z']] = events_df_2['location'].str.split(',', expand=True)
events_df_2[['pass_end_location_x', 'pass_end_location_y']] = events_df_2['pass_end_location'].str.split(',', expand=True)
events_df_2[['carry_end_location_x', 'carry_end_location_y']] = events_df_2['carry_end_location'].str.split(',', expand=True)
events_df_2[['shot_end_location_x', 'shot_end_location_y', 'shot_end_location_z']] = events_df_2['shot_end_location'].str.split(',', expand=True)

# List of columns to drop
drop_cols = ['pass_body_part_id', 'pass_type_id', 'pass_outcome_id', 'clearance_body_part_id',
             'duel_type_id', 'dribble_outcome_id', 'duel_outcome_id', 'interception_outcome_id',
             'pass_technique_id', 'shot_key_pass_id', 'shot_technique_id', 'shot_body_part_id',
             'shot_type_id', 'shot_outcome_id', 'goalkeeper_end_location', 'goalkeeper_position_id',
             'goalkeeper_position_name', 'goalkeeper_type_id', 'goalkeeper_type_name',
             'goalkeeper_outcome_id', 'goalkeeper_outcome_name', 'goalkeeper_technique_id',
             'goalkeeper_technique_name', 'goalkeeper_body_part_id', 'goalkeeper_body_part_name',
             '50_50_outcome_id', 'foul_committed_advantage', 'substitution_outcome_id', 
             'substitution_replacement_id', 'location', 'pass_end_location', 'carry_end_location',
             'shot_end_location']

# Drop columns
for col in drop_cols:
    if col in events_df_2.columns:
        del events_df_2[col]

# Assign a name for DataFrame
events_df_2.name = "events"

del [df_types, drop_cols]


In [36]:
# View DataFrame
#events_df_2.info(verbose = True, show_counts = True)
print(events_df_2.shape)
#events_df_2

(3521, 103)


### 3.6.2 Shot Freeze Frame <a class="anchor" id="section_3_6_2"></a>

In [37]:
# Save new instance
shot_freeze_frame_df_2 = shot_freeze_frame_df

# Remove rows that aren't shots
shot_freeze_frame_df_2 = shot_freeze_frame_df_2[shot_freeze_frame_df_2['shot_freeze_frame'].notna()]

# Create row for each player
shot_freeze_frame_df_2 = shot_freeze_frame_df_2.explode('shot_freeze_frame').reset_index(drop=True)

# Unpack nested dictionary to DataFrame columns
col = shot_freeze_frame_df_2['shot_freeze_frame'].tolist()
col_df = pd.json_normalize(col)

# Join DataFrames together
shot_freeze_frame_df_2 = pd.concat([shot_freeze_frame_df_2, col_df], axis=1)

# Replace '.' from column names with '_'
shot_freeze_frame_df_2.columns = shot_freeze_frame_df_2.columns.str.replace('.','_',  regex = True)

# Create a unique id column
shot_freeze_frame_df_2.insert(0, 'shot_freeze_frame_id', shot_freeze_frame_df_2.event_id.astype(str) +
                                                         shot_freeze_frame_df_2.player_id.astype(str))

# Remove shot_freeze_frame
shot_freeze_frame_df_2.drop('shot_freeze_frame', inplace = True, axis = 1)

# Save copy of shot_freeze_frame_df_2
df_types = shot_freeze_frame_df_2

# Remove square brackets from events_df_2
shot_freeze_frame_df_2 = shot_freeze_frame_df_2.astype(str).replace({"\[":"", "\]":""}, regex = True)

# Assign original dtypes of events_df_2
shot_freeze_frame_df_2 = shot_freeze_frame_df_2.astype(df_types.dtypes.to_dict())

# Assign a name for DataFrame
shot_freeze_frame_df_2.name = "shot_freeze_frame"

del [col, col_df, df_types]

In [38]:
# View DataFrame
#shot_freeze_frame_df_2.info()
print(shot_freeze_frame_df_2.shape)
#shot_freeze_frame_df_2


(298, 9)


## 3.7 Transform Lineups Data <a class="anchor" id="section_3_7"></a>

### 3.7.1 Lineups <a class="anchor" id="section_3_7_1"></a>

In [39]:
# Save new instance
lineups_df_2 = lineups_df

# Drop column events
lineups_df_2 = lineups_df_2.drop(['events'], axis = 1)

# Save formations to a new dataframe
formations_df = lineups_df_2.drop(['lineup'], axis = 1)

# Drop formations from lineups_df
lineups_df_2 = lineups_df_2.drop(['formations'], axis = 1)

# Explode lineup column
lineups_df_2 = lineups_df_2.explode('lineup')

# Normalize lineup column
lineups_norm = pd.json_normalize(lineups_df_2['lineup'])

# Combine the original dataframe and the expl lineup dataframe
lineups_df_2 = pd.concat([lineups_df_2.reset_index(drop = True), lineups_norm.reset_index(drop = True)], axis = 1).drop('lineup', axis = 1)

# Explode positions column
lineups_df_2 = lineups_df_2.explode('positions')

# Normalize positions column
positions_norm = pd.json_normalize(lineups_df_2['positions'])

# Combine the original dataframe and the positions dataframe
lineups_df_2 = pd.concat([lineups_df_2.reset_index(drop = True), positions_norm.reset_index(drop = True)], axis = 1).drop('positions', axis = 1)

# Remove stats columns
lineups_df_2 = lineups_df_2.drop(['stats.own_goals', 'stats.goals', 'stats.assists', 'stats.penalties_scored', 'stats.penalties_missed', 'stats.penalties_saved', 'stats'], axis = 1)

# Change column names
lineups_df_2.rename(columns={'country.id': 'country_id', 'country.name': 'country_name'}, inplace=True)

# Change NaN to -1 in position_id and counterpart_id column
cols = ['position_id', 'counterpart_id']
lineups_df_2[cols] = lineups_df_2[cols].fillna(-1)

# Convert the position_id and counterpart_id columns to integer data type
lineups_df_2[cols] = lineups_df_2[cols].astype('int64')

# Create a unique id column by joining match_id, player_id, position timestamp
lineups_df_2.insert(0, 'match_player_pos_id', np.where(lineups_df_2['from'].isnull(), lineups_df_2['match_id'].astype(str) +
                                                       lineups_df_2['player_id'].astype(str) + '999999',
                                                       lineups_df_2['match_id'].astype(str) +
                                                       lineups_df_2['player_id'].astype(str) +
                                                       lineups_df_2['position_id'].astype(str) +
                                                       pd.to_timedelta(lineups_df_2['from']).dt.total_seconds().div(60).round(2).astype(str).str.replace(
                                                           '.','', regex = False)))

# Convert columns to int type
lineups_df_2 = lineups_df_2.astype({'match_player_pos_id':'int'})

# Convert the newly created match_player_pos_id to integer data type
lineups_df_2['match_player_pos_id'] = lineups_df_2['match_player_pos_id'].astype('int64')

# Assign a name for DataFrame
lineups_df_2.name = 'lineups'

del cols

In [40]:
# View DataFrame
print(lineups_df_2.shape)
#lineups_df_2

(47, 24)


### 3.7.2 Formations <a class="anchor" id="section_3_7_2"></a>

In [41]:
# Save new instance
formations_df_2 = formations_df

# Explode lineup column
formations_df_2 = formations_df_2.explode('formations')

# Normalize lineup column
formations_norm = pd.json_normalize(formations_df_2['formations'])

# Combine the original dataframe and the positions dataframe
formations_df_2 = pd.concat([formations_df_2.reset_index(drop = True), formations_norm.reset_index(drop = True)], axis = 1).drop('formations', axis = 1)

# Create a unique id column by joining match_id, team_id, timestamp
formations_df_2.insert(0, 'match_team_time_id', formations_df_2['match_id'].astype(str) +
                                                       formations_df_2['team_id'].astype(str) +
                                                       formations_df_2['timestamp'].astype(str).str.replace(':','').str.split('.').str[0])

# Convert columns to int type
formations_df_2 = formations_df_2.astype({'match_team_time_id':'int'})

# Assign a name for DataFrame
formations_df_2.name = 'formations'


In [42]:
# View DataFrame
print(formations_df_2.shape)
#formations_df_2

(2, 8)


##################################################################################################################
# 4. Load <a class="anchor" id="chapter4"></a>
##################################################################################################################

## 4.1 Option A: Export to .csv <a class="anchor" id="section_4_1"></a>

In [43]:
# Choose dataframes to export to .csv
dataframes_to_csv = [
                     competitions_df_2, 
                     matches_df_2,
                     team_season_df_2,
                     player_season_df_2,
                     player_match_df_2,
                     team_match_df_2,
                     events_df_2,
                     shot_freeze_frame_df_2, 
                     lineups_df_2, 
                     formations_df_2
                    ]

In [46]:
# Loop through DataFrames
for i in dataframes_to_csv:
    
    # Datetime object containing current date and time
    now = datetime.now()

    # Reformat now (YY/mm/dd H:M:S)
    date_string = now.strftime("%Y%m%d_%H%M%S")
    
    # Name of API endpoint
    endpoint = i.name.capitalize()

    
    ################### Create folder structure
    
     # Create new sub-directory NB. Will fail with FileExistsError if file path exists
    try:
        os.makedirs(f'Data/{endpoint}/')
        print(f"{endpoint} file path created")

    except FileExistsError:
        print(f"{endpoint} file path exists")
        
        # Create new file path string
    new_path = f'/Data/{endpoint}/'

    # Append the new path to the system path
    file_path = str(os.getcwd()) + new_path
    
    
    ################### Create file name

    # Create dictionary to match competition_id to competition_name from matches API
    dict_competition = dict(zip(competitions_df.competition_id, competitions_df.competition_name))

    # Get value of country chosen in section 2.2
    competition_required_int = int(competition_required)
    competition_chosen = dict_competition[competition_required_int].replace(' ', '')
    
    # Create dictionary to match season_id to season_name from matches API
    dict_season = dict(zip(competitions_df.season_id, competitions_df.season_name))

    # Get value of season chosen in section 2.2
    season_required_int = int(season_required)
    season_chosen = dict_season[season_required_int].replace('/', '_')
    
    
    ################### Create new csv for competitions, team season and player season DataFrames
    
    if i is competitions_df_2 or i is team_season_df_2 or i is player_season_df_2:
        
        i.to_csv(file_path + f"{endpoint}_{competition_chosen}_{season_chosen}_({date_string}CET).csv")
        print(f'Create new {endpoint} csv')
    
    else:
    
        ################### Create or append csv for other DataFrames

        # Check if csv already exists
        path = Path(f'{file_path}/{endpoint}_{competition_chosen}_{season_chosen}.csv')
        
        # If no csv exists
        if path.is_file() == False:

            # Create new csv
            print(f'Create new {endpoint} csv')

            i.to_csv(file_path + f'{endpoint}_{competition_chosen}_{season_chosen}.csv', index = False)
            print(f"{endpoint} csv exported")
        
        # If csv exists
        elif path.is_file() == True:
            
            # Append to csv
            print(f'Append to existing {endpoint} csv')

            # Load csv
            df_csv = pd.read_csv(f'{file_path}/{endpoint}_{competition_chosen}_{season_chosen}.csv')

            # Find unique rows
            unique_rows = i[~i.iloc[:, 0].isin(df_csv.iloc[:, 0])]
            
            if i is events_df_2:
                
                # Merge new DataFrame with csv DataFrame
                new_df = pd.concat([df_csv, i])
                
                # Replace csv
                new_df.to_csv(f'{file_path}/{endpoint}_{competition_chosen}_{season_chosen}.csv',
                                 index = False, header = True)
                
            else:
                # Append unique rows to csv
                unique_rows.to_csv(f'{file_path}/{endpoint}_{competition_chosen}_{season_chosen}.csv',
                                  mode = 'a', index = False, header = False)

            del [unique_rows, df_csv]
        
    print("-" * 40)
    
del [now, date_string, endpoint, new_path, file_path, dict_competition,
    competition_required_int, competition_chosen, dict_season, season_required_int,
    season_chosen]

Competitions file path exists
Create new Competitions csv
----------------------------------------
Matches file path exists
Append to existing Matches csv
----------------------------------------
Team_season file path exists
Create new Team_season csv
----------------------------------------
Player_season file path exists
Create new Player_season csv
----------------------------------------
Player_match file path exists
Append to existing Player_match csv
----------------------------------------
Team_match file path exists
Append to existing Team_match csv
----------------------------------------
Events file path exists
Append to existing Events csv
----------------------------------------
Shot_freeze_frame file path exists
Append to existing Shot_freeze_frame csv
----------------------------------------
Lineups file path exists
Append to existing Lineups csv
----------------------------------------
Formations file path exists
Append to existing Formations csv
-------------------------

## 4.2 Option B: Load to layer_1 <a class="anchor" id="section_4_2"></a>

In [47]:
# Database layer_1 credentials
from Credentials.project_credentials import db_layer_1_credentials

# Get layer_1 credentials
layer_1_creds = db_layer_1_credentials()
user_1 = layer_1_creds["user"]
password_1 = layer_1_creds["password"]
host_1 = layer_1_creds["host"]
port_1 = layer_1_creds["port"]
database_1 = layer_1_creds["database"]

# Create a connection to 'layer_1' database
engine_1 = create_engine(f"postgresql://{user_1}:{password_1}@{host_1}:{port_1}/{database_1}")

del [layer_1_creds, user_1, password_1, host_1, port_1, database_1]

In [48]:
#### ADD x3 : events and team_match_df. 

dataframes_to_load = [
                      competitions_df_2, 
                      matches_df_2,
                      team_season_df_2,
                      player_season_df_2,
                      player_match_df_2,
                      team_match_df_2,
                      events_df_2,
                      shot_freeze_frame_df_2, 
                      lineups_df_2, 
                      formations_df_2
                     ]

# Start time
t1 = datetime.now()

# Load Dataframes
for df in dataframes_to_load:
    load_to_db(df, df.name, engine_1)    
    
# End time
t2 = datetime.now()

# Time to load
time = t2 - t1
print(f'Time to Load = {time}')

# Remove variables
del [t1, t2, time]


competitions_store table exists
competitions_store table data updated
----------------------------------------
matches_store table exists
New data appended to matches_store
----------------------------------------
team_season_store table exists
team_season_store table data updated
----------------------------------------
player_season_store table exists
player_season_store table data updated
----------------------------------------
player_match_store table exists
New data appended to player_match_store
----------------------------------------
team_match_store table exists
New data appended to team_match_store
----------------------------------------
events_store table exists
New data appended to events_store
----------------------------------------
shot_freeze_frame_store table exists
New data appended to shot_freeze_frame_store
----------------------------------------
lineups_store table exists
New data appended to lineups_store
----------------------------------------
formations_stor